In [1]:
import time
import requests as req
import re
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
location_webdriver = r"/Applications/chromedriver 2"
driver = webdriver.Chrome(location_webdriver)

In [3]:
def get_page_info(soup, channel_title):
    SOCIAL_BLADE_RANK = []
    SUBSCRIBER_RANK = [] 
    VIDEO_VIEWS_RANK = [] 
    COUNTRY_RANK = [] 
    MUSIC_RANK = []

    for i in (soup.find_all('div', {'id': 'socialblade-user-content' })) :
        rows = i.find_all('div', {'style': 'float: left; width: 900px; height: 150px;' })
        for j in rows :
            cols = j.find_all('div', {'style': 'float: right; width: 720px; padding: 10px; background: #fff; border-bottom: 2px solid #e2e2e2;'})
            for k in cols :
                q = k.find_all('p', {'style': 'font-size: 1.4em; color:#41a200; font-weight: 600;'})
                for index, row in enumerate(q) :
                    if index == 0 :
                        SOCIAL_BLADE_RANK.append(row.text.strip())
                    elif index == 1 : 
                        SUBSCRIBER_RANK.append(row.text.strip())
                    elif index == 2 : 
                        VIDEO_VIEWS_RANK.append(row.text.strip())
                    elif index == 3 : 
                        COUNTRY_RANK.append(row.text.strip())
                    elif index == 4 : 
                        MUSIC_RANK.append(row.text.strip())
                    else :
                        continue


    SUBSCRIBERS_FOR_THE_LAST_30_DAYS = []
    VIDEO_VIEWS_FOR_THE_LAST_30_DAYS = []

    for i in (soup.find_all('div', {'id': 'socialblade-user-content' })) :
        rows = i.find_all('div', {'style': 'float: left; width: 900px; height: 100px; margin-bottom: 10px;' })
        for j in rows :
            cols = j.find_all('p', {'style': 'font-size: 1.4em; color:#666; font-weight: 600; padding-top: 14px; padding-left: 30px; text-align: center;'})
            for index, row in enumerate(cols) :
                    if index == 0 :
                        SUBSCRIBERS_FOR_THE_LAST_30_DAYS.append(row.text.strip())
                    elif index == 1 : 
                        VIDEO_VIEWS_FOR_THE_LAST_30_DAYS.append(row.text.strip())
                    else :
                        continue


    ESTIMATED_MONTHLY_EARNINGS = []

    for i in (soup.find_all('div', {'id': 'socialblade-user-content' })) :
        rows = i.find_all('div', {'style': 'float: left; width: 900px; height: 100px; margin-bottom: 10px;' })
        for j in rows :
            cols = j.find_all('p', {'style': 'font-size: 1.4em; color:#41a200; font-weight: 600; padding-top: 20px;'})
            for index, row in enumerate(cols) :
                    if index == 0 :
                        ESTIMATED_MONTHLY_EARNINGS.append(row.text.strip())
                    else :
                        continue



    ESTIMATED_YEARLY_EARNINGS = []

    for i in (soup.find_all('div', {'id': 'socialblade-user-content' })) :
        rows = i.find_all('div', {'style': 'width: 900px; float: left; font-size: 8pt;' })
        for j in rows :
            cols = j.find_all('p', {'style': 'font-size: 2em; color:#41a200; font-weight: 600; padding-top: 20px;'})
            for index, row in enumerate(cols) :
                    if index == 0 :
                        ESTIMATED_YEARLY_EARNINGS.append(row.text.strip())
                    else :
                        continue
    
    
    headers = ['Channel', 'SOCIAL_BLADE_RANK', 'VIDEO_VIEWS_RANK', 'COUNTRY_RANK ',
       'MUSIC_RANK ', 'ESTIMATED_MONTHLY_EARNINGS ',
       'ESTIMATED_YEARLY_EARNINGS ', 'SUBSCRIBERS_FOR_THE_LAST_30_DAYS ',
       'VIDEO_VIEWS_FOR_THE_LAST_30_DAYS ']
    
    return dict(zip(headers, [channel_title,
                                    SOCIAL_BLADE_RANK,
                                    VIDEO_VIEWS_RANK,
                                    COUNTRY_RANK,
                                    MUSIC_RANK,
                                    ESTIMATED_MONTHLY_EARNINGS,
                                    ESTIMATED_YEARLY_EARNINGS,
                                    SUBSCRIBERS_FOR_THE_LAST_30_DAYS,
                                    VIDEO_VIEWS_FOR_THE_LAST_30_DAYS,
                                    ]))
    


In [4]:
driver.get("https://socialblade.com/youtube/")
search = driver.find_element_by_xpath('//*[@id="SearchInput"]')
search.send_keys('T-Series')

In [5]:
driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/form/div[1]/button/i').click()
time.sleep(5)
driver.get(driver.current_url)
soup = BeautifulSoup(driver.page_source)

In [6]:
pd.DataFrame(get_page_info(soup, 'T-Series'))

,Channel,SOCIAL_BLADE_RANK,VIDEO_VIEWS_RANK,COUNTRY_RANK,MUSIC_RANK,ESTIMATED_MONTHLY_EARNINGS,ESTIMATED_YEARLY_EARNINGS,SUBSCRIBERS_FOR_THE_LAST_30_DAYS,VIDEO_VIEWS_FOR_THE_LAST_30_DAYS
0,T-Series,6th,1st,1st,1st,$585K - $9.4M,$7M - $112.3M,2M 33.3%,2.34B 33.9%


In [7]:
df = pd.read_csv("Top_1000_Youtube_Channels.csv")
df.head()

,Pos,Channel,Views,Subscriber,Vidoes,Published
0,1,T-Series,"157,452,055,214","185,000,000","15,528",2006-03-13
1,2,Cocomelon - Nursery Rhymes,"103,803,532,922","113,000,000",654,2006-09-01
2,3,SET India,"90,768,285,232","107,000,000","59,421",2006-09-20
3,4,WWE,"59,710,652,499","78,500,000","55,466",2007-05-11
4,5,Like Nastya,"58,866,363,302","74,700,000",589,2016-12-06


In [8]:
df_list = []

for channel in df.Channel.values:
    search = driver.find_element_by_xpath('//*[@id="SearchInput"]')
    search.send_keys(channel)
    driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/form/div[1]/button/i').click()
    time.sleep(5)
    driver.get(driver.current_url)
    soup = BeautifulSoup(driver.page_source)
    df_list.append(pd.DataFrame(get_page_info(soup, channel)) )


In [9]:
df_new = pd.concat(df_list)
df_new

,Channel,SOCIAL_BLADE_RANK,VIDEO_VIEWS_RANK,COUNTRY_RANK,MUSIC_RANK,ESTIMATED_MONTHLY_EARNINGS,ESTIMATED_YEARLY_EARNINGS,SUBSCRIBERS_FOR_THE_LAST_30_DAYS,VIDEO_VIEWS_FOR_THE_LAST_30_DAYS
0,T-Series,6th,1st,1st,1st,$585K - $9.4M,$7M - $112.3M,2M 33.3%,2.34B 33.9%
0,Cocomelon - Nursery Rhymes,8th,2nd,1st,1st,$383.1K - $6.1M,$4.6M - $73.5M,1M 0.0%,1.532B 10.1%
0,SET India,7th,3rd,2nd,1st,$593K - $9.5M,$7.1M - $113.9M,2M 33.3%,2.372B 5.1%
0,WWE,27th,5th,4th,1st,$178.3K - $2.9M,$2.1M - $34.2M,700K 46.2%,713.04M 28.6%
0,Like Nastya,28th,7th,5th,2nd,$149.5K - $2.4M,$1.8M - $28.7M,500K 16.7%,597.852M 18.5%
...,...,...,...,...,...,...,...,...,...
0,또봇 V · 메탈리온 [Tobot V · Metalions],"17,495th","1,772nd",187th,589th,$3.1K - $49.6K,$37.2K - $595.3K,--,12.401M 4.7%
0,planbVEVO,"16,205th","1,774th","2,053rd","1,288th",$3.5K - $55.7K,$41.8K - $668.5K,--,13.926M 2.0%
0,TheSoundYouNeed,"14,051st","1,784th",30th,524th,$3.2K - $51.5K,$38.6K - $617.5K,--,12.865M 10%
0,Ch3Thailand Music,"9,183rd","1,755th",81st,798th,$4.2K - $67.6K,$50.7K - $811.3K,20K 0.0%,16.902M 13.5%


In [14]:
df_new.to_csv("SocialBlade.csv", index = False)